In [1]:
import dill

import numpy, scipy, scipy.optimize
import matplotlib
from mpl_toolkits.mplot3d import  Axes3D
from matplotlib import cm
import matplotlib.pyplot as plt

In [2]:
%matplotlib widget

In [3]:
dill.load_session('sessions/Discrete_BS_Q_Learning_vectorized_1-Copy1.db')

In [4]:
print(Q)

defaultdict(<function <lambda> at 0x0000027295A64168>, {(0, 112.5): array([4.87450771, 4.80378965, 4.84665362, 4.76148995, 4.87725285,
       4.83963963, 4.83394449, 4.78486899, 4.79986026, 4.72753211,
       4.81197912]), (1.0, 117.5): array([4.80500181, 4.80565369, 4.79456361, 4.79575328, 4.79434952,
       4.78089792, 4.76997291, 4.82604542, 4.78615459, 4.80981392,
       4.74404773]), (2.0, 122.5): array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), (0, 82.5): array([4.49385579, 4.50648497, 4.50383795, 2.2860277 , 4.35378557,
       4.46447452, 3.14541771, 4.3942399 , 4.37907436, 3.772463  ,
       3.83945412]), (1.0, 82.5): array([4.45462533, 4.2361502 , 4.42821319, 4.2304683 , 4.29817392,
       4.37358031, 4.31269745, 4.20826042, 4.29746974, 4.2560607 ,
       4.24542168]), (2.0, 87.5): array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), (0, 137.5): array([5.41032463, 5.05268963, 5.03859087, 5.0118576 , 5.42361669,
       4.97017819, 5.07078863, 5.06584725, 5.22059137, 5.05295

In [5]:
print(Q.keys())

dict_keys([(0, 112.5), (1.0, 117.5), (2.0, 122.5), (0, 82.5), (1.0, 82.5), (2.0, 87.5), (0, 137.5), (1.0, 137.5), (2.0, 142.5), (0, 107.5), (1.0, 112.5), (2.0, 117.5), (0, 80.0), (1.0, 80.0), (2.0, 80.0), (0, 142.5), (1.0, 152.5), (2.0, 157.5), (0, 87.5), (1.0, 92.5), (2.0, 97.5), (1.0, 147.5), (2.0, 152.5), (0, 127.5), (1.0, 132.5), (0, 92.5), (2.0, 82.5), (0, 122.5), (1.0, 127.5), (2.0, 132.5), (0, 132.5), (0, 147.5), (2.0, 160.0), (1.0, 142.5), (1.0, 107.5), (2.0, 112.5), (1.0, 87.5), (2.0, 92.5), (0, 102.5), (2.0, 107.5), (1.0, 157.5), (1.0, 97.5), (0, 117.5), (1.0, 122.5), (2.0, 127.5), (2.0, 137.5), (1.0, 160.0), (2.0, 102.5), (1.0, 102.5), (0, 97.5), (2.0, 147.5)])


In [6]:
def plot_q_values(Q, actions):
    '''Creates a 3d Wireframe plot of the Q-value function for each state-action pair 
       and adds the predicted action (i.e. argmax_a Q(s,a)
    
    Args:
    :params: Q [dict] A dictionary containing the action-values for each state
    :params: actions [np.array] A np.array containing the possible actions
    '''
    
    
    def fun(x, y, Q, actions, t):
        '''Help function used in plot_q_values'''
        # Returns the Q-values for each state-action pair at time step .
        return np.array([Q[(t,wealth)][encode_action(action, actions)] for action, wealth in zip(x,y)])
    
    times = sorted(list(set([t for t,_ in Q.keys()])))
    for time in times:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        x = actions
        y = np.array(sorted([wealth for t, wealth in Q.keys() if t == time]))
        X, Y = np.meshgrid(x, y)
        zs = np.array(fun(np.ravel(X), np.ravel(Y), Q, actions, time))
        Z = zs.reshape(X.shape)

        # Predicted Actions for each state
        states = [key for key in Q.keys() if key[0] == time]
        predicted_actions = [decode_action(np.argmax(Q[state]), actions) for state in states]
        wealths = [wealth for _, wealth in states]
        predicted_Q_values = [Q[state][np.argmax(Q[state])] for state in states]

        ax.plot_wireframe(X, Y, Z, color="black")

        ax.set_xlabel('investment in risky asset')
        ax.set_ylabel('wealth')
        ax.set_zlabel('Q-values')
        ax.scatter(predicted_actions, wealths, predicted_Q_values, zdir="z", c="red", alpha=1, label="Predicted Actions")
        plt.title("Learned Q-value surface (at t={})".format(time))
        ax.legend()

        plt.show()

In [7]:
plot_q_values(Q, actions)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
def dict_to_list(Q, actions):
    '''Converts the action-value table Q from a dictionary to a list of four numpy arrays.
    
    Args:
    - Q:[dict] Q-Table
    - actions:[np.array] possible risky asset allocations
    
    Returns a list of four np.arrays:
        - time
        - action
        - wealth
        - Q-value
    '''
    
    tData = np.array([t for t,_ in Q.keys()]).repeat(len(actions))
    aData = np.array(list(actions)*(len(tData)//len(actions)))
    vData = np.array([V for _,V in Q.keys()]).repeat(len(actions))
    qData = np.array([values for values in Q.values()]).reshape(len(tData))
    
    # Remove lowest/highest wealth values
    tData = tData[(vData!=min(vData)) & (vData!=max(vData))]
    #tData = tData[vData!=max(vData)]
    aData = aData[(vData!=min(vData)) & (vData!=max(vData))]
    #aData = aData[vData!=max(vData)]
    qData = qData[(vData!=min(vData)) & (vData!=max(vData))]
    #qData = qData[vData!=max(vData)]
    vData = vData[(vData!=min(vData)) & (vData!=max(vData))]
    #vData = vData[vData!=max(vData)]
    
    return [tData, aData, vData, qData]

In [9]:
def ScatterPlot(data):
    #f = plt.figure(figsize=(graphWidth/100.0, graphHeight/100.0), dpi=100)
    f = plt.figure()

    matplotlib.pyplot.grid(True)
    axes = Axes3D(f)
    x_data = data[0]
    y_data = data[1]
    z_data = data[2]

    axes.scatter(x_data, y_data, z_data)

    axes.set_title('Scatter Plot (click-drag with mouse)')
    axes.set_xlabel('X Data')
    axes.set_ylabel('Y Data')
    axes.set_zlabel('Z Data')

    plt.show()
    #plt.close('all') # clean up after using pyplot or else thaere can be memory and process problems

In [36]:
def SurfacePlot(func, data, fittedParameters):
    #f = plt.figure(figsize=(graphWidth/100.0, graphHeight/100.0), dpi=100)
    f = plt.figure()
    
    matplotlib.pyplot.grid(True)
    axes = Axes3D(f)

    x_data = data[0]
    y_data = data[1]
    z_data = data[2]

    xModel = numpy.array(list(set(x_data)))
    yModel = numpy.array(list(set(y_data)))
    X, Y = numpy.meshgrid(xModel, yModel)

    Z = func(numpy.array([1, X, Y]), *fittedParameters)
    #print(Z)

    axes.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=cm.coolwarm, linewidth=1, antialiased=True)

    axes.scatter(x_data, y_data, z_data) # show data along with plotted surface

    axes.set_title('Surface Plot (click-drag with mouse)') # add a title for surface plot
    axes.set_xlabel('') # X axis data label
    axes.set_ylabel('wealth') # Y axis data label
    axes.set_zlabel('Q-Values') # Z axis data label

    plt.show()
    #plt.close('all') # clean up after using pyplot or else thaere can be memory and process problems

In [37]:
data = dict_to_list(Q, actions)

tData = data[0]
aData = data[1]
vData = data[2]
qData = data[3]


#tData = np.array([t for t,_ in Q.keys()]).repeat(len(actions))
#vData = np.array([V for _,V in Q.keys()]).repeat(len(actions))
#aData = np.array(list(actions)*(len(tData)//len(actions)))
#qData = np.array([values for values in Q.values()]).reshape(len(tData))

print(tData)
print(vData)
print(aData)
print(qData)
print(aData.shape)
print(vData.shape)
print(tData.shape)
print(qData.shape)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.

In [26]:
def func(data, a, b, c):
    '''Parametrised action-value function Q in the log-utility case.'''
    
    t   = data[0]
    inv = data[1]
    v   = data[2]
    
    # See master thesis
    z = a * (inv**2) + b * inv + np.log(v) + c * (2-t-1)
    # Q-values for terminal states are zero
    z[t==T]=0
    
    return  z

In [27]:
# Fitting the global (each point in time) paramatrised action value function

initialParameters = [-1,0,0]
fittedParameters, pcov = scipy.optimize.curve_fit(func, [tData, aData, vData], qData, p0 = initialParameters)

print(fittedParameters)

[ 0.02526623 -0.03710839  0.08636059]


In [14]:
data = [aData[tData==1], vData[tData==1], qData[tData==1]]

In [15]:
# Plotting the Q-value function at t=0,1
ScatterPlot([aData[tData==0], vData[tData==0], qData[tData==0]])
ScatterPlot([aData[tData==1], vData[tData==1], qData[tData==1]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
# Plotting the (globally) fitted Q-value function at t=0,1
fittedQValues_0 = func([tData[tData==0], aData[tData==0], vData[tData==0]], *fittedParameters)
fittedQValues_1 = func([tData[tData==1], aData[tData==1], vData[tData==1]], *fittedParameters)

ScatterPlot([aData[tData==0], vData[tData==0], fittedQValues_0])
ScatterPlot([aData[tData==1], vData[tData==1], fittedQValues_1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
print(fittedQValues_0)

[4.80931381 4.80585563 4.80290278 4.80045525 4.79851305 4.79707617
 4.79614462 4.79571839 4.79579749 4.79638191 4.79747165 4.49915888
 4.49570071 4.49274785 4.49030033 4.48835812 4.48692125 4.48598969
 4.48556346 4.48564256 4.48622698 4.48731672 5.00998451 5.00652633
 5.00357348 5.00112595 4.99918375 4.99774687 4.99681532 4.99638909
 4.99646818 4.9970526  4.99814235 4.76385144 4.76039326 4.75744041
 4.75499288 4.75305068 4.7516138  4.75068225 4.75025602 4.75033511
 4.75091953 4.75200928 5.04570259 5.04224441 5.03929156 5.03684403
 5.03490183 5.03346495 5.0325334  5.03210717 5.03218626 5.03277069
 5.03386043 4.55799938 4.55454121 4.55158835 4.54914083 4.54719862
 4.54576175 4.54483019 4.54440396 4.54448306 4.54506748 4.54615722
 4.93447695 4.93101878 4.92806592 4.9256184  4.92367619 4.92223932
 4.92130776 4.92088153 4.92096063 4.92154505 4.9226348  4.61356923
 4.61011106 4.6071582  4.60471068 4.60276847 4.6013316  4.60040004
 4.59997381 4.60005291 4.60063733 4.60172707 4.89447162 4.8910

In [18]:
print(fittedQValues_1)

[4.76643833 4.76298016 4.76002731 4.75757978 4.75563757 4.7542007
 4.75326914 4.75284291 4.75292201 4.75350643 4.75459618 4.41279829
 4.40934012 4.40638726 4.40393974 4.40199753 4.40056066 4.3996291
 4.39920287 4.39928197 4.39986639 4.40095614 4.92362392 4.92016574
 4.91721289 4.91476536 4.91282316 4.91138628 4.91045473 4.9100285
 4.91010759 4.91069201 4.91178176 4.72295322 4.71949505 4.71654219
 4.71409467 4.71215246 4.71071558 4.70978403 4.7093578  4.7094369
 4.71002132 4.71111106 5.0271646  5.02370642 5.02075357 5.01830604
 5.01636384 5.01492696 5.01399541 5.01356918 5.01364827 5.01423269
 5.01532244 4.52720864 4.52375047 4.52079762 4.51835009 4.51640789
 4.51497101 4.51403945 4.51361323 4.51369232 4.51427674 4.51536649
 4.99382818 4.99037    4.98741715 4.98496962 4.98302742 4.98159054
 4.98065899 4.98023276 4.98031185 4.98089627 4.98198602 4.88658265
 4.88312447 4.88017162 4.87772409 4.87578189 4.87434501 4.87341346
 4.87298723 4.87306632 4.87365074 4.87474049 4.84811636 4.84465819

In [38]:
SurfacePlot(func, [aData[tData==1], vData[tData==1], qData[tData==1]], fittedParameters)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
# Fitting the action-value at each time t=0,1 seperately

fittedParameters_0, _ = scipy.optimize.curve_fit(func, [tData[tData==0], aData[tData==0], vData[tData==0]], qData[tData==0], p0=initialParameters)
fittedParameters_1, _ = scipy.optimize.curve_fit(func, [tData[tData==1], aData[tData==1], vData[tData==1]], qData[tData==1], p0=initialParameters)

C:\tools\Anaconda3\envs\env1\lib\site-packages\scipy\optimize\minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


In [20]:
# Plotting the seperately fitted Q-value function at t=0,1

seperatelyFittedQValues_0 = func([tData[tData==0], aData[tData==0], vData[tData==0]], *fittedParameters_0)
seperatelyFittedQValues_1 = func([tData[tData==1], aData[tData==1], vData[tData==1]], *fittedParameters_1)

ScatterPlot([aData[tData==0], vData[tData==0], seperatelyFittedQValues_0])
ScatterPlot([aData[tData==1], vData[tData==1], seperatelyFittedQValues_1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
print(seperatelyFittedQValues_0)

[4.92883497 4.87246659 4.8262722  4.79025178 4.76440535 4.7487329
 4.74323443 4.74790994 4.76275943 4.7877829  4.82298035 4.61868004
 4.56231167 4.51611727 4.48009686 4.45425042 4.43857797 4.4330795
 4.43775501 4.4526045  4.47762797 4.51282542 5.12950567 5.07313729
 5.02694289 4.99092248 4.96507605 4.94940359 4.94390512 4.94858063
 4.96343012 4.98845359 5.02365105 4.8833726  4.82700422 4.78080982
 4.74478941 4.71894298 4.70327052 4.69777205 4.70244756 4.71729705
 4.74232052 4.77751798 5.16522375 5.10885537 5.06266098 5.02664056
 5.00079413 4.98512168 4.9796232  4.98429871 4.9991482  5.02417168
 5.05936913 4.67752054 4.62115217 4.57495777 4.53893736 4.51309092
 4.49741847 4.49192    4.49659551 4.511445   4.53646847 4.57166592
 5.05399811 4.99762974 4.95143534 4.91541493 4.88956849 4.87389604
 4.86839757 4.87307308 4.88792257 4.91294604 4.94814349 4.73309039
 4.67672202 4.63052762 4.59450721 4.56866077 4.55298832 4.54748985
 4.55216536 4.56701485 4.59203832 4.62723577 5.01399278 4.957624

In [22]:
print(seperatelyFittedQValues_1)

[4.76643833 4.77273624 4.77737946 4.78036798 4.78170182 4.78138096
 4.7794054  4.77577516 4.77049022 4.76355059 4.75495627 4.41279829
 4.4190962  4.42373942 4.42672794 4.42806178 4.42774092 4.42576536
 4.42213512 4.41685018 4.40991055 4.40131623 4.92362392 4.92992183
 4.93456504 4.93755357 4.9388874  4.93856654 4.93659099 4.93296074
 4.92767581 4.92073618 4.91214186 4.72295322 4.72925113 4.73389435
 4.73688287 4.7382167  4.73789584 4.73592029 4.73229005 4.72700511
 4.72006548 4.71147116 5.0271646  5.03346251 5.03810572 5.04109425
 5.04242808 5.04210722 5.04013167 5.03650142 5.03121649 5.02427686
 5.01568254 4.52720864 4.53350655 4.53814977 4.5411383  4.54247213
 4.54215127 4.54017572 4.53654547 4.53126053 4.52432091 4.51572658
 4.99382818 5.00012608 5.0047693  5.00775783 5.00909166 5.0087708
 5.00679525 5.003165   4.99788007 4.99094044 4.98234612 4.88658265
 4.89288055 4.89752377 4.9005123  4.90184613 4.90152527 4.89954972
 4.89591947 4.89063454 4.88369491 4.87510058 4.84811636 4.85441

In [23]:
#ScatterPlot([aData[tData==1], vData[tData==1], fittedQValues])
#print(aData[tData==1])
#print(vData[tData==1])
#print(fittedQValues)

In [24]:
SurfacePlot(func, data, fittedParameters)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
func([1, np.array([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]), 100], *fittedParameters)

array([4.60517019, 4.61943362, 4.6311657 , 4.64036641, 4.64703576,
       4.65117375, 4.65278038, 4.65185564, 4.64839954, 4.64241208,
       4.63389326])

In [150]:
[1,1,2].unique()

AttributeError: 'list' object has no attribute 'unique'